In [143]:
from time import perf_counter
import threading
from threading import current_thread, Thread

In [144]:
def Squre(num: float):
    return float(num)* float(num)

def Cube(num: float):
    return float(num) * float(num) * float(num)

In [145]:
st = perf_counter()
[Squre(num) for num in range(100000)]
[Cube(num) for num in range(100000)]
print(perf_counter()-st)


0.17202339999494143


In [146]:
async def calculate_squares(num_range, res_squre):
    res_squre.extend([Squre(num) for num in range(num_range)])

async def calculate_cube(num_range, res_cube):
    res_cube.extend([Cube(num) for num in range(num_range)])

In [147]:
res_cube = []
res_squre = []

In [148]:
range_of_number = 10000
st = perf_counter()
squre_thread = threading.Thread(target=calculate_squares, args=(range_of_number, res_squre))
cube_thread = threading.Thread(target=calculate_cube, args=(range_of_number, res_cube))
squre_thread.start()
cube_thread.start()
squre_thread.join()
cube_thread.join()
print(perf_counter()-st)

0.051572399999713525


C:\Users\TarunKumarChaubey\anaconda3\envs\CVRnD\lib\threading.py:870: RuntimeWarning: coroutine 'calculate_squares' was never awaited
  self._target(*self._args, **self._kwargs)
C:\Users\TarunKumarChaubey\anaconda3\envs\CVRnD\lib\threading.py:870: RuntimeWarning: coroutine 'calculate_cube' was never awaited
  self._target(*self._args, **self._kwargs)


In [149]:
async def calculate_squares(num_range, res_squre):
    res_squre.extend([Squre(num) for num in range(num_range)])

async def calculate_cube(num_range, res_cube):
    res_cube.extend([Cube(num) for num in range(num_range)])

    
range_of_number = 10000
st = perf_counter()
squre_thread = threading.Thread(target=calculate_squares, args=(range_of_number, res_squre))
cube_thread = threading.Thread(target=calculate_cube, args=(range_of_number, res_cube))
squre_thread.start()
cube_thread.start()
squre_thread.join()
cube_thread.join()
print(perf_counter()-st)

0.0028008000226691365


#### Race Condition 

What is Race Condition ?
* It is a bug generated when you do multi-processing. It occurs because two or more threads tries to update the same variable and results into unreliable output. 
* Concurrent accesses to shared resource can lead to race condition.

Thread synchronization Technique : 
    * A common approach is to protect the critical section of code. (Prevent concurrent access.)

In [36]:
class Bus:
    
    def __init__(self, available_seat):
        self.available_seat = available_seat
        
    def BookSeat(self, seat_required_by_user):
        if self.available_seat >= seat_required_by_user:
            print(f"{seat_required_by_user } Seat allocated to User {current_thread().name}")
            self.available_seat -= seat_required_by_user
            
        else:
            print("All Seats are booked")

In [43]:
bus = Bus(2)

t1 = Thread(target=bus.BookSeat, args=(3,), name="AT")
t2 = Thread(target=bus.BookSeat, args=(2,), name="BT")

t1.start()
t2.start()

# t1.join()
# t2.join()

All Seats are booked
2 Seat allocated to User BT


Thread synchronization Technique
Thread synchronization is defined as a mechanism which ensures that two or more concurrent threads do not simultaneously
execute some particular program segment known as critical section. 

We have following thread synchronization techniques.
* Using Locks. 
* Using R-Lock 
* Using Semaphores. Full screer

#### Using Locks

##### Locks in Python
Threading module provides a Lock class to deal with the race conditions. 

Lock has two states : acquire method 
* Locked : - The lock has been acquiredoge tate of code to lockad any thread that makes an attempt to a until it is released. 
* Unlocked : - The lock has not been a acquired by the next thread that makes an attempt

Acquire lock using acquire()
    - Syntax : mylock.acquire()
    
Release lock using release()
    - Syntax : mylock.release()

acquire() method 
* change the state of code to locked.
* other threads have to wait until lock is released by current working thread. 
    - syntax : lock_object.acquire([blocking True], timeout=-1)

In [80]:
from threading import Lock,current_thread
from time import sleep

In [62]:
mylock = Lock()

In [71]:
# both thread called parallal

def task(lock, text):
    for ind in range(5):
        print(text)
    sleep(3)
    
t1 = Thread(target=task, args=(mylock,"T1"))
t2 = Thread(target=task, args=(mylock,"T2"))

t1.start()
t2.start()

T1
T1
T1
T1
T1
T2
T2
T2
T2
T2


In [ ]:
mylock.release()

In [87]:
# adding lock method here

# first T1 get completed then t2 start executing

def task(lock, text):
    lock.acquire()
    print("Stared Working for {}".format(current_thread().name))
    for ind in range(5):
        print(text)
    sleep(5)
    print("Work Completed for {}".format(current_thread().name))
    lock.release()
    
t1 = Thread(target=task, args=(mylock,"T1"), name='T1')
t2 = Thread(target=task, args=(mylock,"T2"), name='T2')

t1.start()
t2.start()

Stared Working for T1
T1
T1
T1
T1
T1
Work Completed for T1
Stared Working for T2
T2
T2
T2
T2
T2
Work Completed for T2


In [49]:
from threading import * 

class Bus : 
    def __init__(self, name, available_seats, lock): 
        self.available_seats = available_seats 
        self.name = name 
        self.lock = lock 
        
    def reserve(self, need_seats):
        self.lock.acquire() 
        print("Available seats are : ", self.available_seats)        
        if self.available_seats >=need_seats:
            nm=current_thread().name
            print(f"{ need_seats} are allocated to {nm}")
            self.available_seats -= need_seats
        else:
            print("Sorry! seats are not avaiable")
        self.lock.release()
        
b1=Bus ("Mahalakshmi travels",2,Lock()) 
t1=Thread(target=b1.reserve, args=(1,), name="Jay")
t2=Thread(target=b1.reserve,args=(1,), name="Raj")

t1.start()
t2.start() 

Available seats are :  2
1 are allocated to Jay
Available seats are :  1
1 are allocated to Raj


#### RLock - Multithreading

In [7]:
def Func1(st=4):
    sleep(st)
    print("Func1 Done")
    
def Func2(st=4):
    sleep(st)
    print("Func1 Done")

RLocks in Python You cannot acquire multiple times using Lock mechanism.
    - By using RLock you can acquire() multiple times. 
    - Rlock is just a modified version of Lock.

Acquire lock using acquire().

    -Syntax : mylock.acquire()

Release lock using release(). 
    - Lock Syntax : mylock.release()

In [52]:
from threading import RLock
# RLock used to mulitple time acquire and release
# rlock keep track running thread detaila and acquire count details while lock not keep these details

class Bus : 
    def __init__(self, name, available_seats, rlock): 
        self.available_seats = available_seats 
        self.name = name 
        self.rlock = rlock 
        
    def reserve(self, need_seats):
        self.rlock.acquire() 
        print("Available seats are : ", self.available_seats)        
        if self.available_seats >=need_seats:
            nm=current_thread().name
            print(f"{ need_seats} are allocated to {nm}")
            self.available_seats -= need_seats
        else:
            print("Sorry! seats are not avaiable")
        print(self.rlock)
        self.rlock.acquire() 
        
b1=Bus ("Mahalakshmi travels",3,RLock()) 
t1=Thread(target=b1.reserve, args=(1,), name="Jay")
t2=Thread(target=b1.reserve,args=(1,), name="Raj")

t1.start()
t2.start() 

Available seats are :  3
1 are allocated to Jay
<locked _thread.RLock object owner=21528 count=1 at 0x000001DFABC91C60>


Lock Vs RLock

Lock - From Main Function

    main function - acquire lock
    called get_first_line
    get_first_line also ask for acquire lock 
    
in this case program get on hold

and RLock Run Program properly

In [59]:
from threading import Lock, RLock
from time import sleep
l=RLock() # code work due to RLock allow multiple acquire and release
# l=Lock() # code not work due to Lock not allow multiple acquire and release

def get_first_line() :
    l.acquire() 
    print("Code for fetching first line")
    l.release() 

def get_second_line(): 
    l.acquire() 
    print("Code for fetching second line") 
    l.release() 
    
def main():
    l.acquire()
    get_first_line()
    get_second_line()
    l.release()

t1=Thread(target=main)
t1.start()

Code for fetching first line
Code for fetching second line


#### Semaphore - Python

In Lock and RLock, at a time only one Thread is allowed to execute. 
but sometimes our requirement is to execute a particular number of Threads at a time.

Semaphore can be used to limit the access to the shared resources with limited capacity.

Ex - If A website has capacity to view only 100 user at time, but user are 1000 then in this next 100 people get change to view website if previous 100 log out from website

from threading import Semaphore

Create an object of Semaphore class. 
    - Syntax : Semaphore()
    
Acquire lock using acquire(). 
    - Syntax : s.acquire()

In [75]:
from threading import Semaphore, BoundedSemaphore
from time import sleep

'''
BoundedSemaphore has exact same feature like Semaphore except BoundedSemaphore has extra capability to handle 
acquire and release count mismatch
'''

obj = Semaphore(value=2)
'''default value=1, means only 1 thread can access variable at time, it behave like Lock and R Lock

But If we want to execute multiple thread at a time, increase value=10 for 10 thread at time
'''



# creating instance 
def display(name): 
    # calling acquire method 
    obj.acquire() 
    for _ in range(3):
        print(f"Hello, I am in { name } ") 
        sleep(2) 
        # calling release method 
    obj.release() 
        
        # creating multiple thread 
t1 = Thread(target = display,args=("Thread - 1",))
t2 = Thread(target = display,args=("Thread - 2",))
t3 = Thread(target = display,args=("Thread - 3",))
t4 = Thread(target = display,args=("Thread - 4",))
t5 = Thread(target = display,args=("Thread - 5",))

t1.start()
t2.start()
t3.start()
t4.start()
t5.start()

Hello, I am in Thread - 1 
Hello, I am in Thread - 2 
Hello, I am in Thread - 1 
Hello, I am in Thread - 2 
Hello, I am in Thread - 2 Hello, I am in Thread - 1 

Hello, I am in Thread - 3 Hello, I am in Thread - 4 

Hello, I am in Thread - 3 Hello, I am in Thread - 4 

Hello, I am in Thread - 3 Hello, I am in Thread - 4 

Hello, I am in Thread - 5 
Hello, I am in Thread - 5 
Hello, I am in Thread - 5 


#### Exception in Multithreading

In [77]:
from threading import Thread, current_thread
from time import sleep 

def display():
    for i in range(4):
        sleep(0.1) 
        print("hello : "+10) 
        
def show():
    for i in range(4):
        print("Hello I'm in {}".format(current_thread().name))
        sleep(0.5)
        
t1=Thread(target=display, name="T1")
t2=Thread(target=show ,name="T2")
t1.start()
t2.start()
t1.join()
t2.join()
for i in range(4):
    print("Bye")

Exception in thread T1:
Traceback (most recent call last):
  File "C:\Users\TarunKumarChaubey\anaconda3\envs\CVRnD\lib\threading.py", line 932, in _bootstrap_inner
    self.run()
  File "C:\Users\TarunKumarChaubey\anaconda3\envs\CVRnD\lib\threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\TarunKumarChaubey\AppData\Local\Temp\ipykernel_17244\445427953.py", line 7, in display
TypeError: can only concatenate str (not "int") to str


Hello I'm in T2
Hello I'm in T2
Hello I'm in T2
Hello I'm in T2
Bye
Bye
Bye
Bye


display() function has error but do not interrupt other thread execution 

What happens for exception in thread? 

The interpreter calls threading.excepthook() 
argument i.e named tuple with 4 arguments 
* the exception class 
* exception instance/value 
* a traceback object 
* Thread name

For main thread -> sys.excepthook 

fot create thread -> threadthreading.excepthook -> sys.excepthook 

In [82]:
from threading import Thread, current_thread
from time import sleep 
import threading

def custom_hook(args):
        print("Exception Occured In Threas")
        print(args[0])
        print(args[1])
        print(args[2])
        print(args[3])

def display():
    for i in range(4):
        sleep(0.1) 
        print("hello : "+10) 
        
def show():
    for i in range(4):
        print("Hello I'm in {}".format(current_thread().name))
        sleep(0.5)

threading.excepthook = custom_hook
t1=Thread(target=display, name="T1")
t2=Thread(target=show ,name="T2")
t1.start()
t2.start()
t1.join()
t2.join()
for i in range(4):
    print("Bye")

Hello I'm in T2
Exception Occured In Threas
<class 'TypeError'>
can only concatenate str (not "int") to str
<Thread(T1, started 23076)>
Hello I'm in T2
Hello I'm in T2
Hello I'm in T2
Bye
Bye
Bye
Bye


#### Thread Communication
* In concurrent programming, sometimes we need to co-ordinate threads
* Threads communicate with each other code-yug signals.

Three ways:-
* By creating event object
* By creating condition object
* By using queue module

event object used to interact between two thread only

    - First, we have to create an event object.
    - Create two threads which will communicate.
    - Put t2 thread in waiting by using wait().
    - Use set() method in/after t1 threads code.

set():-
    - Set the internal flag to true.
    - If flag is True, waiting thread is awakened.
reset():-
    - Reset the internal flag to false.
    - other thread will wait again.

is_set():-
    - Return True if and only if the internal flag is trus
    - ex. if event.is_set()

wait([timeout]):-
    - Calling this function keep on thue untill flaa is set to True.

In [142]:
import threading
import time
e = threading.Event()
#step-01
def task():
    print ("game is started")
    time.sleep(3)
    e.set()

def end():
    e.wait()
    if e.is_set():
        print("code for destroying")
t1 = threading.Thread(target=task)
t2 = threading.Thread(target=end)
t1.start()
t2.start()

game is started
code for destroying
